# 3-Way AI Debate

A structured debate between three LLMs, each with a distinct personality:

- **Alex (GPT-4.1-mini)** — Moderator. Introduces the topic, asks probing questions, keeps things on track.
- **John (Gemini)** — The Optimist. Sees opportunity everywhere, bullish on the future.
- **Peter (Claude)** — The Skeptic. Questions assumptions, plays devil's advocate.

Each model receives a single system prompt defining its role, and a user prompt containing the full
conversation transcript so far.

In [ ]:
# imports

from IPython.display import display, Markdown
from openai import OpenAI
from dotenv import load_dotenv
import os

In [ ]:
# load environment variables from .env file

load_dotenv(override=True)

### Clients

In [ ]:
openai_client = OpenAI()

openrouter_client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

### Models & Personalities

In [ ]:
ALEX_MODEL = "gpt-4.1-mini"
JOHN_MODEL = "google/gemini-2.5-flash-lite"
PETER_MODEL = "anthropic/claude-3.5-haiku"

In [ ]:
# system prompts

alex_system = """You are Alex, the moderator of a debate.
You introduce the topic, ask probing follow-up questions, challenge weak arguments,
and keep the discussion focused. You are fair but push both sides to go deeper.
Keep your responses concise — 2 to 4 sentences max.
You are in a debate with John and Peter."""

john_system = """You are John, a debater who is optimistic and forward-thinking.
You see opportunity and potential in new developments. You back your points
with practical examples and real-world impact.
Keep your responses concise — 3 to 5 sentences max.
You are in a debate with Peter. Alex is the moderator."""

peter_system = """You are Peter, a debater who is skeptical and analytical.
You question assumptions, point out risks, and demand evidence.
You're not negative — you just want to stress-test ideas before buying in.
Keep your responses concise — 3 to 5 sentences max.
You are in a debate with John. Alex is the moderator."""

### The Topic

In [ ]:
# change this to debate a different subject

topic = "Why does it matter right now to learn LLM engineering? Is this the right time, or is it too early / too late?"

In [ ]:
# shared transcript — single source of truth

transcript = []

In [ ]:
# format transcript into a readable string

def format_transcript():
    if not transcript:
        return "(No conversation yet)"
    lines = []
    for entry in transcript:
        lines.append(f"{entry['speaker']}: {entry['text']}")
    return "\n\n".join(lines)

In [ ]:
# Alex call

def call_alex(instruction):
    user_prompt = f"""{instruction}

The conversation so far:
{format_transcript()} 

Respond as Alex the moderator."""
    
    response = openai_client.chat.completions.create(
        model=ALEX_MODEL,
        messages=[
            {"role": "system", "content": alex_system},
            {"role": "user", "content": user_prompt}
        ],
    )

    reply = response.choices[0].message.content
    transcript.append({"speaker": "Alex (Moderator)", "text": reply})
    return reply

In [ ]:
# Call John

def call_john():
    user_prompt = f"""You are John in a debate moderated by Alex.
The conversation so far:
{format_transcript()}
Respond to what just been said, Stay in character as the optimistic and forward-thinking John."""
    
    response = openrouter_client.chat.completions.create(
        model=JOHN_MODEL,
        messages=[
            {"role": "system", "content": john_system},
            {"role": "user", "content": user_prompt}
        ],
    )
    reply = response.choices[0].message.content
    transcript.append({"speaker": "John (Optimist)", "text": reply})
    return reply

In [ ]:
# call Peter

def call_peter():
    user_prompt = f"""You are Peter in a debate moderated by Alex.

The conversation so far:
{format_transcript()}

Respond to what was just said. Stay in character as the skeptic."""

    response = openrouter_client.chat.completions.create(
        model=PETER_MODEL,
        messages=[
            {"role": "system", "content": peter_system},
            {"role": "user", "content": user_prompt}
        ]
    )
    reply = response.choices[0].message.content
    transcript.append({"speaker": "Peter (Skeptic)", "text": reply})
    return reply

In [ ]:
def show(speaker, text):
    display(Markdown(f"### {speaker}\n\n{text}\n\n---"))

### The Debate

In [ ]:
# round 0 — Alex introduces the topic

transcript = []  # reset

opening = call_alex(f"Introduce this debate topic to John and Peter: {topic}")
show("Alex (Moderator)", opening)

In [ ]:
# rounds 1 through 5

for i in range(1, 6):
    display(Markdown(f"## Round {i}"))

    # John responds
    john_reply = call_john()
    show("John (Optimist)", john_reply)

    # Peter responds
    peter_reply = call_peter()
    show("Peter (Skeptic)", peter_reply)

    # Alex moderates
    alex_reply = call_alex("Ask a follow-up question or challenge one of them. Push the debate deeper.")
    show("Alex (Moderator)", alex_reply)

In [ ]:
# Alex gives closing remarks

closing = call_alex("Summarize the key points from both sides and give your closing remarks to end the debate.")
show("Alex (Moderator)", closing)

### Full Transcript

In [ ]:
print(format_transcript())